In [ ]:
import os
import math
import pandas as pd
import math
import os.path
import time
from bitmex import bitmex
# from binance.client import Client
from datetime import timedelta, datetime
from dateutil import parser
from tqdm import notebook #(Optional, used for progress-bars)
import warnings
warnings.filterwarnings("ignore")

In [ ]:
!pip install bitmex

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.7/67.7 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.9/137.9 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 6.2 MB/s eta 0:00:00


In [ ]:
api_key= 'F10qk6gkE31I6c8_kcwMG3Ci'
api_secret = 'E_6pOjNb-LFCDULA2PXyjb-JTHqr-gN6-VqZZImmoOJ_kMOP'

### CONSTANTS
binsizes = {"1m": 1, "5m": 5, "1h": 60, "1d": 1440}
batch_size = 750
client = bitmex(test=False, api_key=api_key, api_secret=api_secret)



def minutes_of_new_data(symbol, kline_size, data, source):
    if len(data) > 0:  old = parser.parse(data["timestamp"].iloc[-1])
    elif source == "binance": old = datetime.strptime('1 Jan 2017', '%d %b %Y')
    elif source == "bitmex": old = client.Trade.Trade_getBucketed(symbol=symbol, binSize=kline_size, count=1, reverse=False).result()[0][0]['timestamp']
    if source == "binance": new = pd.to_datetime(binance_client.get_klines(symbol=symbol, interval=kline_size)[-1][0], unit='ms')
    if source == "bitmex": new = client.Trade.Trade_getBucketed(symbol=symbol, binSize=kline_size, count=1, reverse=True).result()[0][0]['timestamp']
    return old, new


def get_all_bitmex(symbol, kline_size, save = False):
    filename = '%s-%s-data.csv' % (symbol, kline_size)
    if os.path.isfile(filename): 
        data_df = pd.read_csv(filename)
    else: 
        data_df = pd.DataFrame()
    oldest_point, newest_point = minutes_of_new_data(symbol, kline_size, data_df, source = "bitmex")
    delta_min = (newest_point - oldest_point).total_seconds()/60
    available_data = math.ceil(delta_min/binsizes[kline_size])
    rounds = math.ceil(available_data / batch_size)
    
    
    if rounds > 0:
        print('Downloading %d minutes of new data available for %s, i.e. %d instances of %s data in %d rounds.' % (delta_min, symbol, available_data, kline_size, rounds))
        for round_num in notebook.tqdm(range(rounds)):
            time.sleep(1)
            new_time = (oldest_point + timedelta(minutes = round_num * batch_size * binsizes[kline_size]))
            data = client.Trade.Trade_getBucketed(symbol=symbol, binSize=kline_size, count=batch_size, startTime = new_time).result()[0]
            temp_df = pd.DataFrame(data)
            data_df = data_df.append(temp_df)
    data_df.set_index('timestamp', inplace=True)
    if save and rounds > 0: data_df.to_csv(filename)
    print('All caught up..!')
    return data_df

In [ ]:
data = get_all_bitmex("XBTUSD", "5m", save=True)

In [ ]:
data2 = data[data.open.isnull() == False]

data2.head()

,symbol,open,high,low,close,trades,volume,vwap,lastSize,turnover,homeNotional,foreignNotional
timestamp,,,,,,,,,,,,
2015-09-25 12:35:00+00:00,XBTUSD,239.99,239.99,239.99,239.99,1,1,239.990,1.0,239990,0.00240,0.575952
2015-09-25 12:40:00+00:00,XBTUSD,239.99,237.43,237.39,237.39,3,4500,237.410,1500.0,1068345000,10.68345,2536.357877
2015-09-25 12:45:00+00:00,XBTUSD,237.39,237.49,237.37,237.49,2,2000,237.400,500.0,474800000,4.74800,1127.175254
2015-09-25 12:50:00+00:00,XBTUSD,237.49,237.47,237.36,237.45,4,5000,237.432,500.0,1187160000,11.87160,2818.697844
2015-09-25 12:55:00+00:00,XBTUSD,237.45,237.45,237.45,237.45,0,0,NaN,NaN,0,0.00000,0.000000


In [ ]:
data_1m = get_all_bitmex("XBTUSD", "1m", save=True)
data_1m.shape

  0%|          | 0/5341 [00:00<?, ?it/s]

All caught up..!


(4005557, 12)

In [ ]:
pwd

'/content/drive/MyDrive/Colab Notebooks'

In [ ]:
os.chdir('/content/drive/MyDrive/Colab Notebooks')

In [ ]:
data_1m.shape

(4005557, 12)

In [ ]:
data_1m.to_csv("/content/drive/MyDrive/Colab Notebooks/bitmex_data_1m.csv")

In [ ]:
data_1m.iloc[1000000:].head(10)

,symbol,open,high,low,close,trades,volume,vwap,lastSize,turnover,homeNotional,foreignNotional
timestamp,,,,,,,,,,,,
2017-08-19 22:41:00+00:00,XBTUSD,4168.3,4168.3,4161.2,4165.2,62,189495,4166.3195,9260.0,4548387286,45.483873,189495.0
2017-08-19 22:42:00+00:00,XBTUSD,4165.2,4166.5,4161.6,4162.3,36,48804,4162.8507,870.0,1172378403,11.723784,48804.0
2017-08-19 22:43:00+00:00,XBTUSD,4162.3,4162.1,4157.6,4157.6,48,134404,4160.0799,222.0,3230917549,32.309175,134404.0
2017-08-19 22:44:00+00:00,XBTUSD,4157.6,4160.0,4157.0,4159.9,50,146616,4158.3500,1000.0,3525930471,35.259305,146616.0
2017-08-19 22:45:00+00:00,XBTUSD,4159.9,4160.0,4159.9,4159.9,4,12258,4160.0799,1000.0,294660062,2.946601,12258.0
2017-08-19 22:46:00+00:00,XBTUSD,4159.9,4163.5,4159.9,4163.5,65,77651,4162.1577,874.0,1865659328,18.656593,77651.0
2017-08-19 22:47:00+00:00,XBTUSD,4163.5,4164.0,4163.6,4164.0,18,18830,4163.8907,2700.0,452232842,4.522328,18830.0
2017-08-19 22:48:00+00:00,XBTUSD,4164.0,4166.3,4163.9,4166.3,35,98972,4165.4517,1071.0,2376112073,23.761121,98972.0
2017-08-19 22:49:00+00:00,XBTUSD,4166.3,4168.8,4166.2,4168.7,55,56514,4167.1876,50.0,1356196082,13.561961,56514.0


In [ ]:
data_1m.tail()

,symbol,open,high,low,close,trades,volume,vwap,lastSize,turnover,homeNotional,foreignNotional
timestamp,,,,,,,,,,,,
2023-05-08 03:13:00+00:00,XBTUSD,28320.5,28328.5,28320.5,28328.5,12,50000,28322.75,200.0,176536630,1.765366,50000.0
2023-05-08 03:14:00+00:00,XBTUSD,28328.5,28330.0,28329.5,28330.0,11,43500,28329.65,200.0,153549633,1.535496,43500.0
2023-05-08 03:15:00+00:00,XBTUSD,28330.0,28329.5,28320.0,28320.0,51,173600,28326.10,100.0,612862282,6.128623,173600.0
2023-05-08 03:16:00+00:00,XBTUSD,28320.0,28322.5,28289.5,28304.0,154,1151700,28313.05,3300.0,4067744000,40.677440,1151700.0
2023-05-08 03:17:00+00:00,XBTUSD,28304.0,28314.0,28301.5,28304.5,52,138400,28308.30,400.0,488903245,4.889032,138400.0


In [ ]:
data_test = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/bitmex_data_1m.csv")

print(data_test.shape)
print(data_test.tail())

(4005557, 13)
                         timestamp  symbol     open     high      low  \
4005552  2023-05-08 03:13:00+00:00  XBTUSD  28320.5  28328.5  28320.5   
4005553  2023-05-08 03:14:00+00:00  XBTUSD  28328.5  28330.0  28329.5   
4005554  2023-05-08 03:15:00+00:00  XBTUSD  28330.0  28329.5  28320.0   
4005555  2023-05-08 03:16:00+00:00  XBTUSD  28320.0  28322.5  28289.5   
4005556  2023-05-08 03:17:00+00:00  XBTUSD  28304.0  28314.0  28301.5   

           close  trades   volume      vwap  lastSize    turnover  \
4005552  28328.5      12    50000  28322.75     200.0   176536630   
4005553  28330.0      11    43500  28329.65     200.0   153549633   
4005554  28320.0      51   173600  28326.10     100.0   612862282   
4005555  28304.0     154  1151700  28313.05    3300.0  4067744000   
4005556  28304.5      52   138400  28308.30     400.0   488903245   

         homeNotional  foreignNotional  
4005552      1.765366          50000.0  
4005553      1.535496          43500.0  
4005554  